From https://medium.com/@gahogg/a-gentle-introduction-to-pyspark-b4e9a06199b3

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
titanic_df = spark.read.csv("titanic/train.csv", header = True, inferSchema = True)

In [9]:
titanic_df.count()

891

In [5]:
titanic_df.head(5)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C'),
 Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S'),
 Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S'),
 Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.05, Cabin=None, Embarked='S')]

In [6]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
titanic_df.select('PassengerID','Survived').limit(5)

PassengerID,Survived
1,0
2,1
3,1
4,1
5,0


In [10]:
titanic_df.where(titanic_df.Survived == 1).count()

342

In [11]:
titanic_df.where((titanic_df.Age > 25) & (titanic_df.Survived ==1)).count()

166

In [13]:
survived_over_25_df = titanic_df.where((titanic_df.Age > 25) & (titanic_df.Survived == 1)) # Same as before
survived_over_25_df.select('Name').limit(5)

Name
"Cumings, Mrs. Joh..."
"Heikkinen, Miss. ..."
"Futrelle, Mrs. Ja..."
"Johnson, Mrs. Osc..."
"Bonnell, Miss. El..."


## Aggregation 

In [14]:
# Aggregation! 
# Already have count() in use 
# Other common aggregations are min, max, and sum

titanic_df.agg({'Fare':'avg'})

avg(Fare)
32.2042079685746


In [17]:
titanic_df.groupby("Pclass", "Survived").agg({'Fare':'count'}).orderBy('Pclass', 'Survived', ascending = True)

Pclass,Survived,count(Fare)
1,0,80
1,1,136
2,0,97
2,1,87
3,0,372
3,1,119


## User Defined Functions 

In [19]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def round_float_down(x):
    return int(x)

round_float_down_udf = udf(round_float_down, IntegerType())

In [21]:
# titanic_df.select('PassengerId', 'Fare', round_float_down_udf('Fare').alias('Fare Rounded Down')).limit(5)

trim = titanic_df.select('PassengerId', 'Fare')
# , round_float_down_udf('Fare').alias('Fare Rounded Down')).limit(5)

In [22]:
# THIS BREAKS - looks like a formatting issue? 

trim.select('PassengerId', round_float_down_udf('Fare')).limit(5)

Py4JJavaError: An error occurred while calling o275.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 54.0 failed 1 times, most recent failure: Lost task 0.0 in stage 54.0 (TID 40) (LAPTOP-92MN1OM8.lan executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$anonfun$

Py4JJavaError: An error occurred while calling o275.getRowsToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 55.0 failed 1 times, most recent failure: Lost task 0.0 in stage 55.0 (TID 41) (LAPTOP-92MN1OM8.lan executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$ano